# Calling R from Julia
R is unbeatable in terms of the scope of statictical stuff offered therein. When time is scarce to code lines of GARCH estimation code or systems of VAR equations, a good option is to transfer data to R and let it do the magic. The (registered) package needed is `RCall`.

In [1]:
# Pkg.add("RCall")
using RCall
include("printmat.jl")
using Plots
gr()

  likely near In[1]:2
  likely near In[1]:2


Plots.GRBackend()

There are several ways of interacting with R in Julia, all are more or less equally straightforward. We will stick to treating R as a black box: put in the data, press a button of required label and extract the result. Full and easy-to-understand documentation is available [here](http://juliainterop.github.io/RCall.jl/stable/index.html).

## I/O
Say, we have the following arrays in Julia:

In [17]:
x = randn(40);
y = randn(20).^2;

### to R
Feed to R is done using `@rput`: a variable with the same name will appear in the global environment of R.

In [3]:
# load two arrays to R
@rput x y;

### from R
Retrieval from R is done with `@rget`: a variable with the same name will appear in the environment of Julia.

In [4]:
# let's change x to see that it can be fetched in the original form from R
x = x * 0.0

@rget x

println("\nx should not contain only zeros!\n")
print(x)


x should not contain only zeros!

[-0.731547, -0.322383, 1.5005, 2.45943, -0.597634, 1.25031, 1.37533, 0.750675, -0.468836, 0.126609, 0.660369, 1.08415, -0.593475, 1.47635, -0.68182, 0.121801, 0.541033, -0.0641964, 0.836766, -1.16461, 0.440686, -0.915271, 0.3806, -0.0593146, -0.527779, 0.145068, 1.08603, 0.644788, -0.358337, 1.14389, -1.06098, -0.00448768, 0.702035, -0.594741, 1.08904, -0.223463, 0.398676, 0.700708, -0.585892, -1.55115]

## Calling R functions
is easiest done with macro `R` immediately followed by the command string. The output is an `RObject`, which is a Julia wrapper type around an R object.

In [5]:
# summary stats of `x` with built-in function (make sure variable `x`
#     exists in R!)
R"summary(x)"

RCall.RObject{RCall.RealSxp}
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.5511 -0.5423  0.1358  0.2102  0.7722  2.4594 


In [6]:
# run a t-test on `x`
R"t.test(x)"

RCall.RObject{RCall.VecSxp}

	One Sample t-test

data:  x
t = 1.519, df = 39, p-value = 0.1368
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.06970643  0.49015226
sample estimates:
mean of x 
0.2102229 



### Example
I could not find a linear filter in Julia to create an AR(1) series:

$x_t = 0.9 x_{t-1} + \varepsilon_t$

R offers this (after 30 seconds of googling) via:

In [18]:
# make sure `x` is in R
x_ar = R"filter(x, c(0.9), method = 'recursive')";
print(x_ar)

RCall.RObject{RCall.RealSxp}
Time Series:
Start = 1 
End = 40 
Frequency = 1 
 [1] -0.7353761 -0.7800113 -0.5259384 -0.5923219 -0.8904255 -0.9802599
 [7] -1.0382085 -1.1938132 -1.4558969  0.8658535  0.1348591 -1.5148393
[13] -0.8385020 -1.2060845 -2.2163716 -3.6976418 -4.2169202 -3.7837485
[19] -4.8672891 -4.6688556 -4.4010451 -4.7925378 -4.9923989 -4.6248064
[25] -4.7470023 -5.2637021 -5.3855680 -4.3587785 -3.8880342 -3.1319125
[31] -2.9485905 -5.2370793 -5.0854575 -3.7318425 -4.1051935 -4.0400226
[37] -4.0620126 -4.4209104 -4.2774624 -5.1307538


Variables from Julia environment can be parsed using '$' as below:

In [19]:
# define AR coefficient
rho = 0.9

# `x` and `rho` will be parsed from Julia's environment
x_ar = R"filter($x, c($rho), method = 'recursive')";
print(x_ar)

RCall.RObject{RCall.RealSxp}
Time Series:
Start = 1 
End = 40 
Frequency = 1 
 [1]  0.17815024 -0.12154686  0.09417130 -1.11362277 -0.37603407 -1.25184835
 [7] -0.41862502 -0.34885411 -1.57679721 -1.53467372 -2.04508880 -2.09197489
[13] -1.45676519 -0.71811854 -0.80069840 -0.24906901 -0.60064072 -2.50283780
[19] -3.60618391 -3.75132379 -4.96096118 -3.66889715 -1.64435617 -1.42416061
[25] -1.15700225 -0.95223928 -0.68024768  0.90827917  2.45071035  1.75520322
[31]  5.01260999  2.90797022  3.36942264  3.20563263  2.85651423  0.80650974
[37]  1.07249430  1.15746922  1.13456095  0.09520046


`RObject`, as output by the functions above, can be converted back to a familiar type:

In [20]:
# convert `x_ar` to 1-dimensional array of floats
x_ar = convert(Array{Float64, 1}, x_ar);
plot(x_ar)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,186.498 53.5433,197.571 67.7165,189.601 81.8898,234.226 96.063,206.974 110.236,239.333 124.409,208.548 138.583,205.97 152.756,251.34 166.929,249.783 
 181.102,268.642 195.276,270.375 209.449,246.905 223.622,219.613 237.795,222.664 251.969,202.283 266.142,215.273 280.315,285.555 294.488,326.322 308.661,331.684 
 322.835,376.378 337.008,328.639 351.181,253.836 365.354,245.7 379.528,235.829 393.701,228.264 407.874,218.214 422.047,159.521 436.22,102.531 450.394,128.229 
 464.567,7.87402 478.74,85.6363 492.913,68.5866 507.087,74.6383 521.26,87.5375 535.433,163.281 549.606,153.454 563.78,150.314 577.953,151.16 592.126,189.563 
 
 "/>
 
 
 
 
 y1

## OLS in R
has tons of values as output, including exotic stuff. Let's simulate a linear dependency as follows:


$y_t = 1 + 2 x_t + u_t,$

$u_t = 0.9 u_{t-1} + \varepsilon_t,$

$ \varepsilon \sim i.i.d. $

In [21]:
# simulate x
x = randn(40);

# simulate y with autocorrelated residuals from above
y = 1 + x*2 + x_ar;
scatter(x, y)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

OLS (with a constant) using R interface is done like that:
```
mod <- lm(y ~ x)
```
Extraction of statistics from this `mod` object in R is simple - google it!

In [12]:
# put variables into R
@rput x y;

# run estimation and get its summary
R"mod <- lm(y ~ x)";
R"sr <- summary(mod)";

# extract coefficient estimates and their covariance matrix
coef = R"coef(sr)";
vcv = R"vcov(sr)";

print(coef)
print(vcv)

RCall.RObject{RCall.RealSxp}
            Estimate Std. Error   t value     Pr(>|t|)
(Intercept) 3.187735  0.2553007 12.486197 5.077371e-15
x           2.080563  0.2921792  7.120846 1.688780e-08
RCall.RObject{RCall.RealSxp}
            (Intercept)          x
(Intercept)  0.06517846 0.03606906
x            0.03606906 0.08536869


## Foreign libraries
requries extra packages be installed **in this session** of JuliaBox (you will have to reinstall them every time you star a fresh session). Below we construct a function that can calculate robust covariance of OLS-estimated coefficients using R's `sandwich` package, and have to install the package (and its dependecies) first.

In [13]:
# some essential setting for installation to be possible
R"""
options(repos='http://cran.rstudio.com/')
"""

# package 'zoo' which 'sandwich' relies upon
R"""
install.packages("zoo")
"""

# 'sandwich' itself
R"""
install.packages("sandwich")
"""

gcc -std=gnu99 -I/usr/share/R/include -DNDEBUG -I../inst/include     -fpic  -g -O2 -fstack-protector --param=ssp-buffer-size=4 -Wformat -Werror=format-security -D_FORTIFY_SOURCE=2 -g  -c coredata.c -o coredata.o
gcc -std=gnu99 -I/usr/share/R/include -DNDEBUG -I../inst/include     -fpic  -g -O2 -fstack-protector --param=ssp-buffer-size=4 -Wformat -Werror=format-security -D_FORTIFY_SOURCE=2 -g  -c init.c -o init.o
gcc -std=gnu99 -I/usr/share/R/include -DNDEBUG -I../inst/include     -fpic  -g -O2 -fstack-protector --param=ssp-buffer-size=4 -Wformat -Werror=format-security -D_FORTIFY_SOURCE=2 -g  -c lag.c -o lag.o
gcc -std=gnu99 -shared -L/usr/lib/R/lib -Wl,-Bsymbolic-functions -Wl,-z,relro -o zoo.so coredata.o init.o lag.o -L/usr/lib/R/lib -lR


* installing *source* package 'zoo' ...
** package 'zoo' successfully unpacked and MD5 sums checked
** libs
installing to /usr/local/lib/R/site-library/zoo/libs
** R
** demo
** inst
** preparing package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded
* DONE (zoo)
(as 'lib' is unspecified)
trying URL 'http://cran.rstudio.com/src/contrib/zoo_1.8-0.tar.gz'
Content type 'application/x-gzip' length 839729 bytes (820 KB)
downloaded 820 KB


The downloaded source packages are in
	'/tmp/Rtmp3mfQt9/downloaded_packages'
* installing *source* package 'sandwich' ...
** package 'sandwich' successfully unpacked and MD5 sums checked
** R
** data
** inst
** preparing package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded
* DONE (sandwich)
(as 'lib' is unspecified)
trying URL 'http://cran.rstudio.co

RCall.RObject{RCall.NilSxp}
NULL


In [14]:
# here's the function
"""
Estimate linear regression y ~ x

Parameters
----------
x : Array
    regressor
y : Array
    response
addconstant : Bool
    true if constant should be added to `x`
hac : Bool
    true for robust (HAC) covariance matrix of regressors

Returns
-------
(coef, vcv) : Tuple
    of coefficients, covariance matrix
"""
function rols(x::Array, y::Array; addconstant::Bool=false, hac::Bool=false)

    # put `x` and `y` into R environment
    @rput x y

    # construct lm formula
    formula = string("mod <- lm(y ~ x", addconstant ? "" : "-1", ")")

    # evaluate
    reval(formula)

    # extract stuff:
    #   coefficients
    reval("coef <- coef(mod)")
    #   vcv
    if hac
        reval("require(sandwich)")
        reval("vcv <- vcovHAC(mod)")
    else
        reval("vcv <- vcov(mod)")
    end

    # retrieve to julia (local) environment
    @rget coef vcv

    return convert(Array{Float64, 1}, coef), convert(Array{Float64, 2}, vcv)
end

rols

regress `y` that we simulated above on `x`, show covariance matrices with and without HAC:

In [22]:
# regressions
coef, vcv_simpl = rols(x, y, addconstant=true, hac=false);
coef, vcv_hac = rols(x, y, addconstant=true, hac=true);

println("\nCovariance of estimates without HAC\n")
printmat(vcv_simpl)
println("\nCovariance of estimates with HAC\n")
printmat(vcv_hac)


Covariance of estimates without HAC

     0.116    -0.018
    -0.018     0.106


Covariance of estimates with HAC

     0.293     0.008
     0.008     0.127

